In [1]:
import torch
import torch.nn as nn

import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torchvision.transforms.functional as F
from torch.utils.data import DataLoader

from torch.cuda.amp import autocast, GradScaler
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm
import time

from timm.data import Mixup
from timm.utils import ModelEmaV3
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy
import transformers

from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torchsummary import summary

from convnext_v1 import load_convNext_v1
from convnext_v2 import load_convNext
import math
import warnings
from torch.optim.lr_scheduler import _LRScheduler

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CosineWarmupScheduler(_LRScheduler):
    def __init__(self, optimizer, num_warmup_steps, num_training_steps, num_cycles=0.5, min_lr=1e-6, last_epoch=-1, verbose=False):
        self.num_warmup_steps = num_warmup_steps
        self.num_training_steps = num_training_steps
        self.num_cycles = num_cycles
        self.min_lr = min_lr
        self.base_lrs = [group['lr'] for group in optimizer.param_groups]
        super().__init__(optimizer, last_epoch, verbose)

    def get_lr(self):
        if not self._get_lr_called_within_step:
            warnings.warn("To get the last learning rate computed by the scheduler, "
                          "please use `get_last_lr()`.", UserWarning)
        
        lrs = []
        for base_lr in self.base_lrs:
            if self.last_epoch < self.num_warmup_steps:
                # Linear warmup
                lr = (base_lr - self.min_lr) * self.last_epoch / max(1, self.num_warmup_steps) + self.min_lr
            else:
                # Cosine annealing
                progress = (self.last_epoch - self.num_warmup_steps) / max(1, self.num_training_steps - self.num_warmup_steps)
                lr = self.min_lr + (base_lr - self.min_lr) * 0.5 * (1 + math.cos(math.pi * self.num_cycles * 2.0 * progress))
            lrs.append(lr)
        return lrs

In [3]:
model_v1 = load_convNext_v1()

In [4]:
# 총 파라미터 수 계산
total_params = sum(p.numel() for p in model_v1.parameters())

# 학습 가능한 파라미터 수 계산
trainable_params = sum(p.numel() for p in model_v1.parameters() if p.requires_grad)

print('='*80)
print(f"\nTotal Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}\n")
print('='*80)


Total Parameters: 27,897,028
Trainable Parameters: 27,897,028



In [5]:
# Transforms 정의하기
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.6,1), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=1., scale=(0.02, 0.33)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

data_dir = '../../data/sports'
batch_size = 512

train_path = data_dir+'/train'
valid_path = data_dir+'/valid'
test_path = data_dir+'/test'

# dataset load
train_data = ImageFolder(train_path, transform=train_transform)
valid_data = ImageFolder(valid_path, transform=test_transform)
test_data = ImageFolder(test_path, transform=test_transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

device = 'cuda:3'
max_norm = 3.0 

model_v1.to(device)

model_ema = None
ema_active = True
if ema_active:
    ema_decay = 0.9998
    model_ema = ModelEmaV3(
        model_v1,
        decay=ema_decay,
    )
    print(f"Using EMA with decay = {ema_decay}")

model_path = ''

mixup = True
if mixup :
    mixup_fn = Mixup(mixup_alpha=.8, 
                    cutmix_alpha=1., 
                    prob=1., 
                    switch_prob=0.5, 
                    mode='batch',
                    label_smoothing=.1,
                    num_classes=100)
    
    criterion = SoftTargetCrossEntropy()
else :
    criterion = LabelSmoothingCrossEntropy(.1)
    
criterion = nn.CrossEntropyLoss(label_smoothing=0.)

epochs = 100

optimizer = optim.AdamW(model_v1.parameters(), lr=4e-3, weight_decay=0.05)
warmup_steps = int(len(train_loader)*(epochs)*0.1)
train_steps = len(train_loader)*(epochs)
scheduler = CosineWarmupScheduler(optimizer, 
                                num_warmup_steps=warmup_steps, 
                                num_training_steps=train_steps,
                                num_cycles=0.5,
                                min_lr=1e-7)

Using EMA with decay = 0.9998


/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [6]:
training_time = 0
losses = []
val_losses = []
lrs = []
best_loss = float('inf')

# GradScaler 초기화
scaler = GradScaler()

for i in range(epochs // 100):
    for epoch in range(100):
        model_v1.train()
        start_time = time.time()
        running_loss = 0.0
        pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1 + i*100}")
        
        for _, data in pbar:
            inputs, labels = data[0].to(device), data[1].to(device)
            inputs, labels = mixup_fn(inputs, labels)
            optimizer.zero_grad()

            # AutoCast 적용
            with autocast():
                outputs = model_v1(inputs)
                loss = criterion(outputs, labels)
                
            # 스케일링된 그라디언트 계산
            scaler.scale(loss).backward()

            # 그라디언트 클리핑 전에 스케일링 제거
            scaler.unscale_(optimizer)
            clip_grad_norm_(model_v1.parameters(), max_norm=max_norm)

            # 옵티마이저 스텝 및 스케일러 업데이트
            scaler.step(optimizer)
            scaler.update()
            
            # EMA 모델 업데이트
            if model_ema is not None:
                model_ema.update(model_v1)
                
            scheduler.step()
                
            lr = optimizer.param_groups[0]["lr"]
            lrs.append(lr)
            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        losses.append(epoch_loss)        

        model_v1.eval()
        val_loss = 0.0
        with torch.no_grad():
            for data in valid_loader:
                inputs, labels = data[0].to(device), data[1].to(device)
                outputs = model_v1(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
        val_loss /= len(valid_loader)
        val_losses.append(val_loss)
        
        # 모델 저장
        total_loss = val_loss + epoch_loss
        if total_loss < best_loss:
            best_loss = total_loss
            model_save = False
            if model_save:
                torch.save(model_v1.state_dict(), model_path)

        epoch_duration = time.time() - start_time
        training_time += epoch_duration
        
        text = f'\tLoss: {epoch_loss:.4f}, Val_Loss: {val_loss:.4f}, Total Mean Loss: {total_loss/2:.4f}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
        
        if model_save:
            text += f' - model saved!'
            model_save = False

        print(text)

    # 예측 수행 및 레이블 저장
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_v1(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 혼동 행렬 생성
    cm = confusion_matrix(all_labels, all_preds)

    # 예측과 실제 레이블
    y_true = all_labels  # 실제 레이블
    y_pred = all_preds  # 모델에 의해 예측된 레이블

    # 전체 데이터셋에 대한 정확도
    accuracy = accuracy_score(y_true, y_pred)

    # 평균 정밀도, 리콜, F1-Score ('weighted')
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

    # 판다스 데이터프레임으로 결과 정리
    performance_metrics = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
        'Value': [accuracy, precision, recall, f1_score]
    })

    # 데이터프레임 출력
    print(f"\n[{i*100+100} epoch result]\n", performance_metrics)

Epoch 1: 100%|██████████| 27/27 [00:56<00:00,  2.08s/it]


	Loss: 4.4774, Val_Loss: 4.0328, Total Mean Loss: 4.2551, LR: 0.00040008999999999997, Duration: 57.10 sec


Epoch 2: 100%|██████████| 27/27 [00:55<00:00,  2.05s/it]


	Loss: 4.3636, Val_Loss: 3.9938, Total Mean Loss: 4.1787, LR: 0.0008000799999999999, Duration: 56.24 sec


Epoch 3: 100%|██████████| 27/27 [00:55<00:00,  2.06s/it]


...

	Loss: 2.1196, Val_Loss: 0.7607, Total Mean Loss: 1.4401, LR: 4.971777682864596e-06, Duration: 56.02 sec


Epoch 99: 100%|██████████| 27/27 [00:55<00:00,  2.04s/it]


	Loss: 2.0866, Val_Loss: 0.7598, Total Mean Loss: 1.4232, LR: 1.3183155031594304e-06, Duration: 56.12 sec


Epoch 100: 100%|██████████| 27/27 [00:55<00:00,  2.04s/it]


	Loss: 2.1089, Val_Loss: 0.7597, Total Mean Loss: 1.4343, LR: 1e-07, Duration: 56.03 sec

[100 epoch result]
       Metric     Value
0   Accuracy  0.876000
1  Precision  0.896143
2     Recall  0.876000
3   F1 Score  0.872440


In [7]:
model_v1.cpu()
del model_v1

In [8]:
model_v2 = load_convNext()

# 총 파라미터 수 계산
total_params = sum(p.numel() for p in model_v2.parameters())

# 학습 가능한 파라미터 수 계산
trainable_params = sum(p.numel() for p in model_v2.parameters() if p.requires_grad)

print('='*80)
print(f"\nTotal Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}\n")
print('='*80)


Total Parameters: 27,943,396
Trainable Parameters: 27,943,396



In [9]:
model_v2.to(device)

model_ema = None
ema_active = True
if ema_active:
    ema_decay = 0.9998
    model_ema = ModelEmaV3(
        model_v2,
        decay=ema_decay,
    )
    print(f"Using EMA with decay = {ema_decay}")

model_path = ''

mixup = True
if mixup :
    mixup_fn = Mixup(mixup_alpha=.8, 
                    cutmix_alpha=1., 
                    prob=1., 
                    switch_prob=0.5, 
                    mode='batch',
                    label_smoothing=.1,
                    num_classes=100)
    
    criterion = SoftTargetCrossEntropy()
else :
    criterion = LabelSmoothingCrossEntropy(.1)
    
criterion = nn.CrossEntropyLoss(label_smoothing=0.)

epochs = 100

optimizer = optim.AdamW(model_v2.parameters(), lr=4e-3, weight_decay=0.05)
warmup_steps = int(len(train_loader)*(epochs)*0.1)
train_steps = len(train_loader)*(epochs)
scheduler = CosineWarmupScheduler(optimizer, 
                                num_warmup_steps=warmup_steps, 
                                num_training_steps=train_steps,
                                num_cycles=0.5,
                                min_lr=1e-7)

Using EMA with decay = 0.9998


/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [10]:
training_time = 0
losses = []
val_losses = []
lrs = []
best_loss = float('inf')

# GradScaler 초기화
scaler = GradScaler()

for i in range(epochs // 100):
    for epoch in range(100):
        model_v2.train()
        start_time = time.time()
        running_loss = 0.0
        pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1 + i*100}")
        
        for _, data in pbar:
            inputs, labels = data[0].to(device), data[1].to(device)
            inputs, labels = mixup_fn(inputs, labels)
            optimizer.zero_grad()

            # AutoCast 적용
            with autocast():
                outputs = model_v2(inputs)
                loss = criterion(outputs, labels)
                
            # 스케일링된 그라디언트 계산
            scaler.scale(loss).backward()

            # 그라디언트 클리핑 전에 스케일링 제거
            scaler.unscale_(optimizer)
            clip_grad_norm_(model_v2.parameters(), max_norm=max_norm)

            # 옵티마이저 스텝 및 스케일러 업데이트
            scaler.step(optimizer)
            scaler.update()
            
            # EMA 모델 업데이트
            if model_ema is not None:
                model_ema.update(model_v2)
                
            scheduler.step()
                
            lr = optimizer.param_groups[0]["lr"]
            lrs.append(lr)
            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        losses.append(epoch_loss)        

        model_v2.eval()
        val_loss = 0.0
        with torch.no_grad():
            for data in valid_loader:
                inputs, labels = data[0].to(device), data[1].to(device)
                outputs = model_v2(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
        val_loss /= len(valid_loader)
        val_losses.append(val_loss)
        
        # 모델 저장
        total_loss = val_loss + epoch_loss
        if total_loss < best_loss:
            best_loss = total_loss
            model_save = False
            if model_save:
                torch.save(model_v2.state_dict(), model_path)

        epoch_duration = time.time() - start_time
        training_time += epoch_duration
        
        text = f'\tLoss: {epoch_loss:.4f}, Val_Loss: {val_loss:.4f}, Total Mean Loss: {total_loss/2:.4f}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
        
        if model_save:
            text += f' - model saved!'
            model_save = False

        print(text)

    # 예측 수행 및 레이블 저장
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_v2(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 혼동 행렬 생성
    cm = confusion_matrix(all_labels, all_preds)

    # 예측과 실제 레이블
    y_true = all_labels  # 실제 레이블
    y_pred = all_preds  # 모델에 의해 예측된 레이블

    # 전체 데이터셋에 대한 정확도
    accuracy = accuracy_score(y_true, y_pred)

    # 평균 정밀도, 리콜, F1-Score ('weighted')
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

    # 판다스 데이터프레임으로 결과 정리
    performance_metrics = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
        'Value': [accuracy, precision, recall, f1_score]
    })

    # 데이터프레임 출력
    print(f"\n[{i*100+100} epoch result]\n", performance_metrics)

Epoch 1: 100%|██████████| 27/27 [01:06<00:00,  2.46s/it]


	Loss: 4.4953, Val_Loss: 3.9957, Total Mean Loss: 4.2455, LR: 0.00040008999999999997, Duration: 67.57 sec


Epoch 2: 100%|██████████| 27/27 [01:06<00:00,  2.46s/it]


	Loss: 4.3383, Val_Loss: 3.8066, Total Mean Loss: 4.0724, LR: 0.0008000799999999999, Duration: 67.57 sec


Epoch 3: 100%|██████████| 27/27 [01:06<00:00,  2.47s/it]


...

	Loss: 2.1632, Val_Loss: 0.7543, Total Mean Loss: 1.4588, LR: 4.971777682864596e-06, Duration: 67.64 sec


Epoch 99: 100%|██████████| 27/27 [01:06<00:00,  2.46s/it]


	Loss: 2.1685, Val_Loss: 0.7549, Total Mean Loss: 1.4617, LR: 1.3183155031594304e-06, Duration: 67.57 sec


Epoch 100: 100%|██████████| 27/27 [01:06<00:00,  2.46s/it]


	Loss: 2.0983, Val_Loss: 0.7549, Total Mean Loss: 1.4266, LR: 1e-07, Duration: 67.38 sec

[100 epoch result]
       Metric     Value
0   Accuracy  0.894000
1  Precision  0.912238
2     Recall  0.894000
3   F1 Score  0.891306


## SSH with FCMAE

In [3]:
from model.fcmae import convnextv2_fcmae_tiny

model = convnextv2_fcmae_tiny()

/opt/conda/lib/python3.10/site-packages/MinkowskiEngine-0.5.4-py3.10-linux-x86_64.egg/MinkowskiEngine/__init__.py:36: UserWarning: The environment variable `OMP_NUM_THREADS` not set. MinkowskiEngine will automatically set `OMP_NUM_THREADS=16`. If you want to set `OMP_NUM_THREADS` manually, please export it on the command line before running a python script. e.g. `export OMP_NUM_THREADS=12; python your_program.py`. It is recommended to set it below 24.
  warnings.warn(


In [4]:
# Transforms 정의하기
train_transform = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=(0.2, 1.0), interpolation=3),  # 3 is bicubic
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

data_dir = '../../data/sports/'
batch_size = 800
train_path = data_dir

# dataset load
train_data = ImageFolder(train_path, transform=train_transform)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=True)

len(train_loader)
device = 'cuda:0'
model.to(device)

model_path = '../../model/convnext/fcmae.pt'

epochs = 500
optimizer = optim.AdamW(model.parameters(), lr=1.5e-4, weight_decay=0.05, betas=(0.9, 0.95))

warmup_steps = int(len(train_loader)*(epochs)*0.1)
train_steps = len(train_loader)*(epochs)
scheduler = CosineWarmupScheduler(optimizer, 
                                num_warmup_steps=50, 
                                num_training_steps=500,
                                num_cycles=0.5,
                                min_lr=1e-7)

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [ ]:

training_time = 0
losses = []
val_losses = []
lrs = []
best_loss = float('inf')

for epoch in range(epochs):
    model.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for _, data in pbar:
        optimizer.zero_grad()
        
        samples= data[0].to(device)
        loss, _, _ = model(samples, mask_ratio=0.6)
        
        loss.backward()
        optimizer.step()
            
        lr = optimizer.param_groups[0]["lr"]
        lrs.append(lr)
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)        
    
    # 모델 저장 로직 조정
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        if epoch > (epochs // 2) :
            torch.save(model.state_dict(), model_path)
            model_saved_text = ' - model saved!'
        else :
            model_saved_text = ' - model save pass'
    else:
        model_saved_text = ''
    
    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss:,.4f}, LR: {lr}, Duration: {epoch_duration:.2f} sec{model_saved_text}'
    print(text)

    # 에폭마다 스케줄러 업데이트
    scheduler.step()

## FineTuning after ssh

In [3]:
from model.convnextv2 import load_convNext

model = load_convNext(droppath=0.2)

In [4]:
model.state_dict().keys()

odict_keys(['stem.stem_conv.weight', 'stem.stem_conv.bias', 'stem.stem_ln.weight', 'stem.stem_ln.bias', 'downsample_layers.0.stem_conv.weight', 'downsample_layers.0.stem_conv.bias', 'downsample_layers.0.stem_ln.weight', 'downsample_layers.0.stem_ln.bias', 'downsample_layers.1.ds_ln0.weight', 'downsample_layers.1.ds_ln0.bias', 'downsample_layers.1.ds_conv1.weight', 'downsample_layers.1.ds_conv1.bias', 'downsample_layers.2.ds_ln1.weight', 'downsample_layers.2.ds_ln1.bias', 'downsample_layers.2.ds_conv2.weight', 'downsample_layers.2.ds_conv2.bias', 'downsample_layers.3.ds_ln2.weight', 'downsample_layers.3.ds_ln2.bias', 'downsample_layers.3.ds_conv3.weight', 'downsample_layers.3.ds_conv3.bias', 'stages.0.0.dwconv.weight', 'stages.0.0.dwconv.bias', 'stages.0.0.layernorm.weight', 'stages.0.0.layernorm.bias', 'stages.0.0.pwconv1.weight', 'stages.0.0.pwconv1.bias', 'stages.0.0.grn.gamma', 'stages.0.0.grn.beta', 'stages.0.0.pwconv2.weight', 'stages.0.0.pwconv2.bias', 'stages.0.1.dwconv.weight',

In [5]:
model.state_dict()['stem.stem_conv.weight'][0]

tensor([[[-0.0210, -0.1009,  0.1258, -0.1308],
         [ 0.0649,  0.0201, -0.0583,  0.0681],
         [ 0.0280,  0.0370,  0.0578, -0.0648],
         [-0.1016, -0.1418,  0.1152,  0.1145]],

        [[-0.1182, -0.0415,  0.0019,  0.0749],
         [-0.0851,  0.1249, -0.0309,  0.0025],
         [ 0.0188,  0.1149,  0.1096,  0.0237],
         [-0.1322,  0.0144,  0.1397,  0.0282]],

        [[ 0.1233,  0.1423, -0.0962,  0.0821],
         [ 0.0732,  0.0843,  0.0378, -0.0136],
         [ 0.0488,  0.0995,  0.0273,  0.0825],
         [-0.0682, -0.0409,  0.0648, -0.0143]]])

In [6]:
model.state_dict()['stages.0.0.dwconv.weight'][0]

tensor([[[-2.6829e-02,  4.3303e-02,  1.1755e-01,  9.1654e-02,  4.6168e-02,
          -2.0700e-02,  1.3269e-02],
         [ 9.6401e-02,  5.3274e-02, -3.8076e-02,  4.9968e-02,  8.6762e-04,
           6.1436e-02,  1.4115e-01],
         [-1.4044e-01, -1.1432e-01,  2.9335e-02,  1.0103e-01, -1.3687e-02,
          -7.5368e-02, -1.1560e-01],
         [-3.3938e-03,  6.5108e-02, -1.6344e-02,  1.4442e-02, -9.3134e-02,
          -1.2409e-01, -1.5133e-02],
         [-1.3989e-01, -8.8871e-02, -4.0106e-02,  5.0907e-02, -1.2918e-01,
          -2.8023e-02,  5.5662e-02],
         [ 1.3996e-01,  1.3011e-01,  6.6979e-05,  8.9666e-02,  1.3147e-01,
          -2.3832e-02, -8.9970e-02],
         [-3.1620e-02,  7.8192e-02, -7.8324e-02,  1.3465e-01, -1.2800e-01,
          -1.2070e-01,  1.4094e-01]]])

In [7]:
from collections import OrderedDict

def remap_checkpoint_keys(ckpt):
    new_ckpt = OrderedDict()
    for k, v in ckpt.items():
        if k.startswith('encoder'):
            k = '.'.join(k.split('.')[1:]) # remove encoder in the name
        if k.endswith('kernel'):
            k = '.'.join(k.split('.')[:-1]) # remove kernel in the name
            new_k = k + '.weight'
            if len(v.shape) == 3: # resahpe standard convolution
                kv, in_dim, out_dim = v.shape
                ks = int(math.sqrt(kv))
                new_ckpt[new_k] = v.permute(2, 1, 0).\
                    reshape(out_dim, in_dim, ks, ks).transpose(3, 2)
            elif len(v.shape) == 2: # reshape depthwise convolution
                kv, dim = v.shape
                ks = int(math.sqrt(kv))
                new_ckpt[new_k] = v.permute(1, 0).\
                    reshape(dim, 1, ks, ks).transpose(3, 2)
            continue
        elif 'ln' in k or 'linear' in k:
            k = k.split('.')
            k.pop(-2) # remove ln and linear in the name
            new_k = '.'.join(k)
        else:
            new_k = k
        new_ckpt[new_k] = v

    # reshape grn affine parameters and biases
    for k, v in new_ckpt.items():
        if k.endswith('bias') and len(v.shape) != 1:
            new_ckpt[k] = v.reshape(-1)
        elif 'grn' in k:
            new_ckpt[k] = v.unsqueeze(0).unsqueeze(1)
    return new_ckpt

def load_state_dict(model, state_dict, prefix='', ignore_missing="relative_position_index"):
    missing_keys = []
    unexpected_keys = []
    error_msgs = []
    # copy state_dict so _load_from_state_dict can modify it
    metadata = getattr(state_dict, '_metadata', None)
    state_dict = state_dict.copy()
    if metadata is not None:
        state_dict._metadata = metadata

    def load(module, prefix=''):
        local_metadata = {} if metadata is None else metadata.get(
            prefix[:-1], {})
        module._load_from_state_dict(
            state_dict, prefix, local_metadata, True, missing_keys, unexpected_keys, error_msgs)
        for name, child in module._modules.items():
            if child is not None:
                load(child, prefix + name + '.')

    load(model, prefix=prefix)

    warn_missing_keys = []
    ignore_missing_keys = []
    for key in missing_keys:
        keep_flag = True
        for ignore_key in ignore_missing.split('|'):
            if ignore_key in key:
                keep_flag = False
                break
        if keep_flag:
            warn_missing_keys.append(key)
        else:
            ignore_missing_keys.append(key)

    missing_keys = warn_missing_keys

    if len(missing_keys) > 0:
        print("Weights of {} not initialized from pretrained model: {}".format(
            model.__class__.__name__, missing_keys))
    if len(unexpected_keys) > 0:
        print("Weights from pretrained model not used in {}: {}".format(
            model.__class__.__name__, unexpected_keys))
    if len(ignore_missing_keys) > 0:
        print("Ignored weights of {} not initialized from pretrained model: {}".format(
            model.__class__.__name__, ignore_missing_keys))
    if len(error_msgs) > 0:
        print('\n'.join(error_msgs))

In [8]:
pretrain_path = '../../model/convnext/fcmae.pt'
checkpoint_model = torch.load(pretrain_path, map_location='cpu')

state_dict = model.state_dict()
for k in ['head.weight', 'head.bias']:
    if k in checkpoint_model and checkpoint_model[k].shape != state_dict[k].shape:
        print(f"Removing key {k} from head of pretrained checkpoint")
        del checkpoint_model[k]


# remove decoder weights
checkpoint_model_keys = list(checkpoint_model.keys())
for k in checkpoint_model_keys:
    if 'decoder' in k or 'mask_token'in k or \
        'proj' in k or 'pred' in k:
        print(f"Removing key {k} from decoder of pretrained checkpoint")
        del checkpoint_model[k]

checkpoint_model = remap_checkpoint_keys(checkpoint_model)
load_state_dict(model, checkpoint_model, prefix='')

Removing key mask_token from decoder of pretrained checkpoint
Removing key proj.weight from decoder of pretrained checkpoint
Removing key proj.bias from decoder of pretrained checkpoint
Removing key decoder.0.dwconv.weight from decoder of pretrained checkpoint
Removing key decoder.0.dwconv.bias from decoder of pretrained checkpoint
Removing key decoder.0.layernorm.weight from decoder of pretrained checkpoint
Removing key decoder.0.layernorm.bias from decoder of pretrained checkpoint
Removing key decoder.0.pwconv1.weight from decoder of pretrained checkpoint
Removing key decoder.0.pwconv1.bias from decoder of pretrained checkpoint
Removing key decoder.0.grn.gamma from decoder of pretrained checkpoint
Removing key decoder.0.grn.beta from decoder of pretrained checkpoint
Removing key decoder.0.pwconv2.weight from decoder of pretrained checkpoint
Removing key decoder.0.pwconv2.bias from decoder of pretrained checkpoint
Removing key pred.weight from decoder of pretrained checkpoint
Removing

In [9]:
model.state_dict()['stem.stem_conv.weight'][0]

tensor([[[ 0.2945,  0.7202, -0.3333, -0.2289],
         [-0.5944,  0.4452,  0.9049, -0.2540],
         [ 0.4562, -0.8116,  1.4202,  0.4324],
         [ 0.1777,  0.3165, -1.3547,  0.2427]],

        [[-1.5330, -1.2280, -0.4023, -0.5692],
         [-0.5322, -0.0219,  0.3301, -1.0079],
         [ 0.4587, -0.0214, -0.6747, -0.7620],
         [ 0.1962, -0.4949, -0.0375, -0.0354]],

        [[-0.8695,  0.9264, -0.2321, -1.5825],
         [-0.4034, -0.4807,  0.2547, -0.3684],
         [-0.6111, -1.2094,  0.8702,  0.9287],
         [ 1.1989,  0.0221, -0.5370,  0.5360]]])

In [20]:
model.state_dict()['stages.0.0.dwconv.weight'][0]

tensor([[[ 1.2617, -0.2949, -0.1866,  0.1731,  0.0968, -0.2891, -0.9204],
         [-0.5181,  0.9795, -0.8359,  0.5205, -0.4570,  0.2246, -0.1680],
         [-0.5283, -0.1686, -0.2800,  1.1797, -1.3213,  0.7119, -1.5332],
         [-0.2485, -0.1683,  1.0381,  0.2788,  0.3088,  0.2258, -0.1799],
         [ 1.6299,  0.7847,  0.8198, -1.2764, -0.3535, -1.3438,  0.1346],
         [-0.9390,  0.4802,  0.1530,  0.2759, -0.5771, -0.9058,  0.6665],
         [ 0.7075,  0.3372, -0.4517, -0.3804,  0.4553,  0.9985,  0.3589]]],
       device='cuda:4')

In [19]:
model = model.float()

In [13]:
# 총 파라미터 수 계산
total_params = sum(p.numel() for p in model.parameters())

# 학습 가능한 파라미터 수 계산
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print('='*80)
print(f"\nTotal Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}\n")
print('='*80)


Total Parameters: 27,943,396
Trainable Parameters: 27,943,396



In [21]:
# Transforms 정의하기
train_transform = transforms.Compose([
    transforms.TrivialAugmentWide(interpolation=F.InterpolationMode.BICUBIC),
    transforms.RandomResizedCrop(224, scale=(0.6,1), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

data_dir = '../../data/sports'
batch_size = 256

train_path = data_dir+'/train'
valid_path = data_dir+'/valid'
test_path = data_dir+'/test'

# dataset load
train_data = ImageFolder(train_path, transform=train_transform)
valid_data = ImageFolder(valid_path, transform=test_transform)
test_data = ImageFolder(test_path, transform=test_transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [22]:
device = 'cuda:4'
max_norm = 3.0 

model.to(device)

model_ema = None
ema_active = True
if ema_active:
    ema_decay = 0.999
    model_ema = ModelEmaV3(
        model,
        decay=ema_decay,
    )
    print(f"Using EMA with decay = {ema_decay}")

model_path = ''

mixup = True
if mixup :
    mixup_fn = Mixup(mixup_alpha=.8, 
                    cutmix_alpha=1., 
                    prob=1., 
                    switch_prob=0.5, 
                    mode='batch',
                    label_smoothing=.1,
                    num_classes=100)
    
    criterion = SoftTargetCrossEntropy()
else :
    criterion = LabelSmoothingCrossEntropy(.1)
    
criterion = nn.CrossEntropyLoss(label_smoothing=0.)

Using EMA with decay = 0.999


### LayerWise Learning Decay

In [23]:
# LLRD
layer_names = []
for i, (name, params) in enumerate(model.named_parameters()):
    layer_names.append(name)
    
layer_names.reverse()

lr      = 8e-3  
lr_mult = 0.9  
weight_decay = 0.05 

param_groups = []
prev_group_name = layer_names[0].split('.')[0]

for idx, name in enumerate(layer_names):
    
    cur_group_name = name.split('.')[0]
    
    if cur_group_name != prev_group_name:
        lr *= lr_mult
    prev_group_name = cur_group_name
    
    # print(f"{idx}: {name}'s lr={lr}")
    
    param_groups += [{'params': [ p for n, p in model.named_parameters() if n == name and p.requires_grad],
                      'lr' : lr,
                      'weight_decay': weight_decay}]

In [24]:
epochs = 100

optimizer = optim.AdamW(param_groups)
warmup_steps = int(len(train_loader)*(epochs)*0.1)
train_steps = len(train_loader)*(epochs)
scheduler = CosineWarmupScheduler(optimizer, 
                                num_warmup_steps=warmup_steps, 
                                num_training_steps=train_steps,
                                num_cycles=0.5,
                                min_lr=1e-7)

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [30]:
training_time = 0
losses = []
val_losses = []
lrs = []
best_loss = float('inf')
# model_save = False
loss_sampling = []


# GradScaler 초기화
scaler = GradScaler()

for i in range(epochs // 100):
    for epoch in range(100):
        model.train()
        start_time = time.time()
        running_loss = 0.0
        pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1 + i*100}")
        
        for _, data in pbar:
            inputs, labels = data[0].to(device), data[1].to(device)
            inputs, labels = mixup_fn(inputs, labels)
            optimizer.zero_grad()

            # AutoCast 적용
            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
            # 스케일링된 그라디언트 계산
            scaler.scale(loss).backward()

            # 그라디언트 클리핑 전에 스케일링 제거
            scaler.unscale_(optimizer)
            clip_grad_norm_(model.parameters(), max_norm=max_norm)

            # 옵티마이저 스텝 및 스케일러 업데이트
            scaler.step(optimizer)
            scaler.update()
            
            # EMA 모델 업데이트
            if model_ema is not None:
                model_ema.update(model)
                
            scheduler.step()
                
            lr = optimizer.param_groups[0]["lr"]
            lrs.append(lr)
            running_loss += loss.item()
            loss_sampling.append(loss.item())

        epoch_loss = running_loss / len(train_loader)
        losses.append(epoch_loss)        

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for data in valid_loader:
                inputs, labels = data[0].to(device), data[1].to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
        val_loss /= len(valid_loader)
        val_losses.append(val_loss)
        
        # 모델 저장
        total_loss = val_loss + epoch_loss
        if total_loss < best_loss:
            best_loss = total_loss
            model_save = False
            if model_save:
                torch.save(model.state_dict(), model_path)

        epoch_duration = time.time() - start_time
        training_time += epoch_duration
        
        text = f'\tLoss: {epoch_loss:.4f}, Val_Loss: {val_loss:.4f}, Total Mean Loss: {total_loss/2:.4f}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
        
        if model_save:
            text += f' - model saved!'
            model_save = False

        print(text)

    # 예측 수행 및 레이블 저장
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 혼동 행렬 생성
    cm = confusion_matrix(all_labels, all_preds)

    # 예측과 실제 레이블
    y_true = all_labels  # 실제 레이블
    y_pred = all_preds  # 모델에 의해 예측된 레이블

    # 전체 데이터셋에 대한 정확도
    accuracy = accuracy_score(y_true, y_pred)

    # 평균 정밀도, 리콜, F1-Score ('weighted')
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

    # 판다스 데이터프레임으로 결과 정리
    performance_metrics = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
        'Value': [accuracy, precision, recall, f1_score]
    })

    # 데이터프레임 출력
    print(f"\n[{i*100+100} epoch result]\n", performance_metrics)

Epoch 1:   0%|          | 0/53 [00:00<?, ?it/s]

Epoch 1: 100%|██████████| 53/53 [01:21<00:00,  1.54s/it]


NameError: name 'model_save' is not defined

In [29]:
running_loss

nan

In [27]:
loss

tensor(4.7050, device='cuda:4')

In [16]:
training_time = 0
losses = []
val_losses = []
lrs = []
best_loss = float('inf')
model_save = False

for i in range(epochs // 100):
    for epoch in range(100):
        model.train()
        start_time = time.time()
        running_loss = 0.0
        pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1 + i*100}")
        
        for _, data in pbar:
            inputs, labels = data[0].to(device), data[1].to(device)
            inputs, labels = mixup_fn(inputs, labels)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
                
            loss.backward()
            # 그라디언트 클리핑 적용
            clip_grad_norm_(model.parameters(), max_norm=max_norm)
            optimizer.step()
            
            # EMA 모델 업데이트, 필요한 경우
            if model_ema is not None:
                model_ema.update(model)
                
            scheduler.step()
                
            lr = optimizer.param_groups[0]["lr"]
            lrs.append(lr)
            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        losses.append(epoch_loss)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for data in valid_loader:
                inputs, labels = data[0].to(device), data[1].to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
        val_loss /= len(valid_loader)
        val_losses.append(val_loss)
        
        # 모델 저장 조건 수정
        total_loss = val_loss + epoch_loss
        if total_loss < best_loss:
            best_loss = total_loss
            # torch.save(model.state_dict(), model_path)
            model_save = True
            save_text = ' - model saved!'
        else:
            save_text = ''

        epoch_duration = time.time() - start_time
        training_time += epoch_duration
        
        text = f'\tLoss: {epoch_loss:.4f}, Val_Loss: {val_loss:.4f}, Total Mean Loss: {total_loss/2:.4f}, LR: {lr}, Duration: {epoch_duration:.2f} sec{save_text}'
        print(text)

    # 예측 수행 및 레이블 저장
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 혼동 행렬 생성
    cm = confusion_matrix(all_labels, all_preds)

    # 예측과 실제 레이블
    y_true = all_labels  # 실제 레이블
    y_pred = all_preds  # 모델에 의해 예측된 레이블

    # 전체 데이터셋에 대한 정확도
    accuracy = accuracy_score(y_true, y_pred)

    # 평균 정밀도, 리콜, F1-Score ('weighted')
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

    # 판다스 데이터프레임으로 결과 정리
    performance_metrics = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
        'Value': [accuracy, precision, recall, f1_score]
    })

    # 데이터프레임 출력
    print(f"\n[{i*100+100} epoch result]\n", performance_metrics)

Epoch 1: 100%|██████████| 53/53 [01:41<00:00,  1.92s/it]


	Loss: 4.5950, Val_Loss: 4.1453, Total Mean Loss: 4.3702, LR: 0.00080009, Duration: 103.15 sec - model saved!


Epoch 2: 100%|██████████| 53/53 [01:39<00:00,  1.88s/it]


	Loss: 4.4743, Val_Loss: 4.0018, Total Mean Loss: 4.2381, LR: 0.0016000800000000003, Duration: 100.79 sec - model saved!


Epoch 3: 100%|██████████| 53/53 [01:40<00:00,  1.89s/it]


...

	Loss: 2.5952, Val_Loss: 0.8477, Total Mean Loss: 1.7215, LR: 9.843677163216202e-06, Duration: 103.13 sec


Epoch 99: 100%|██████████| 53/53 [01:40<00:00,  1.90s/it]


	Loss: 2.5665, Val_Loss: 0.8481, Total Mean Loss: 1.7073, LR: 2.5366614649679064e-06, Duration: 102.18 sec


Epoch 100: 100%|██████████| 53/53 [01:40<00:00,  1.90s/it]


	Loss: 2.5940, Val_Loss: 0.8485, Total Mean Loss: 1.7212, LR: 1e-07, Duration: 101.85 sec

[100 epoch result]
       Metric     Value
0   Accuracy  0.858000
1  Precision  0.885246
2     Recall  0.858000
3   F1 Score  0.852777


In [11]:
training_time = 0
losses = []
val_losses = []
lrs = []
best_loss = float('inf')
model_save = False

for i in range(epochs // 100):
    for epoch in range(100):
        model.train()
        start_time = time.time()
        running_loss = 0.0
        pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1 + i*100}")
        
        for _, data in pbar:
            inputs, labels = data[0].to(device), data[1].to(device)
            inputs, labels = mixup_fn(inputs, labels)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
                
            loss.backward()
            # 그라디언트 클리핑 적용
            clip_grad_norm_(model.parameters(), max_norm=max_norm)
            optimizer.step()
            
            # EMA 모델 업데이트, 필요한 경우
            if model_ema is not None:
                model_ema.update(model)
                
            scheduler.step()
                
            lr = optimizer.param_groups[0]["lr"]
            lrs.append(lr)
            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        losses.append(epoch_loss)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for data in valid_loader:
                inputs, labels = data[0].to(device), data[1].to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
        val_loss /= len(valid_loader)
        val_losses.append(val_loss)
        
        # 모델 저장 조건 수정
        total_loss = val_loss + epoch_loss
        if total_loss < best_loss:
            best_loss = total_loss
            # torch.save(model.state_dict(), model_path)
            model_save = True
            save_text = ' - model saved!'
        else:
            save_text = ''

        epoch_duration = time.time() - start_time
        training_time += epoch_duration
        
        text = f'\tLoss: {epoch_loss:.4f}, Val_Loss: {val_loss:.4f}, Total Mean Loss: {total_loss/2:.4f}, LR: {lr}, Duration: {epoch_duration:.2f} sec{save_text}'
        print(text)

    # 예측 수행 및 레이블 저장
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 혼동 행렬 생성
    cm = confusion_matrix(all_labels, all_preds)

    # 예측과 실제 레이블
    y_true = all_labels  # 실제 레이블
    y_pred = all_preds  # 모델에 의해 예측된 레이블

    # 전체 데이터셋에 대한 정확도
    accuracy = accuracy_score(y_true, y_pred)

    # 평균 정밀도, 리콜, F1-Score ('weighted')
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

    # 판다스 데이터프레임으로 결과 정리
    performance_metrics = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
        'Value': [accuracy, precision, recall, f1_score]
    })

    # 데이터프레임 출력
    print(f"\n[{i*100+100} epoch result]\n", performance_metrics)

Epoch 1: 100%|██████████| 53/53 [01:26<00:00,  1.63s/it]


	Loss: 4.5563, Val_Loss: 4.1654, Total Mean Loss: 4.3608, LR: 0.00080009, Duration: 87.62 sec - model saved!


Epoch 2: 100%|██████████| 53/53 [01:26<00:00,  1.63s/it]


	Loss: 4.3917, Val_Loss: 3.9178, Total Mean Loss: 4.1548, LR: 0.0016000800000000003, Duration: 87.55 sec - model saved!


Epoch 3: 100%|██████████| 53/53 [01:25<00:00,  1.61s/it]


...

Epoch 73: 100%|██████████| 53/53 [01:25<00:00,  1.61s/it]


	Loss: 2.4625, Val_Loss: 0.9573, Total Mean Loss: 1.7099, LR: 0.0016489383800927227, Duration: 86.77 sec


Epoch 74: 100%|██████████| 53/53 [01:25<00:00,  1.61s/it]


	Loss: 2.4015, Val_Loss: 0.9411, Total Mean Loss: 1.6713, LR: 0.0015374348817711334, Duration: 86.72 sec


Epoch 75:  53%|█████▎    | 28/53 [00:45<00:40,  1.61s/it]